In [ ]:
import pinocchio as pin
import example_robot_data
import numpy as np

In [ ]:
z1 = example_robot_data.load("z1")

# Load the urdf model
model = z1.model

# Create data required by the algorithms
data = model.createData()

In [ ]:
# Helpers

# function for damped pseudo-inverse
def damped_pseudoinverse(jac, l = 0.01):
    m, n = jac.shape
    if n >= m:
        return jac.T @ np.linalg.inv(jac @ jac.T + l*l*np.eye(m))
    return np.linalg.inv(jac.T @ jac + l*l*np.eye(n)) @ jac.T

# function to compute FK for all frames/joints
def fk_all(model, data, q):
    pin.forwardKinematics(model,data,q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
# Sample a random configuration as the target
pin.seed(np.random.randint(low=0, high=999999999)) # give a random seed to pinocchio
qd = pin.randomConfiguration(model)
print('qd:', qd.T)

In [ ]:
# Let's check our FK

# zero configuration!
fk_all(model, data, np.zeros_like(qd))
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")

# target configuration!
fk_all(model, data, qd)
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")

In [ ]:
# We want to compute IK for our tip ("link06")
frame_id = model.getFrameId("link06")

In [ ]:
# first of all, let's compute the transformation matrix
fk_all(model, data, qd)
T_wd = data.oMf[frame_id].copy()

print(T_wd)

In [ ]:
# Newton's algorithm for IK
error = np.ones((6, 1))
# We start without any guess, all in the initial configuration!
q = z1.q0.copy()

step = 0.2
it = 0
success = False
while True:
    # first we need to compute FK
    fk_all(model, data, q)
    # We now need to get our current transformation matrix
    T_wb = data.oMf[frame_id]
    # We know compute the error
    error = pin.log(T_wb.actInv(T_wd)).vector # log(T_wb^-1 T_wd), error in local frame
    # error = pin.log(T_wd.act(T_wb.inverse())).vector # log(T_wd T_wb^-1), error in world frame
    # print("#" + str(it) + ":", error.T)
    if (np.abs(error) <= 1e-6).all():
        success = True
        break
    if it > 1000:
        success = False
        break
    # Compute Jacobian
    J = pin.computeFrameJacobian(model, data, q, frame_id, pin.ReferenceFrame.LOCAL) # Jacobian in local frame
    # J = pin.computeFrameJacobian(model, data, q, frame_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED) # Jacobian in world frame
    J_pinv = damped_pseudoinverse(J)
    v = J_pinv @ error
    # Compute next q given the velocity
    q = pin.integrate(model, q, v * step)
    it += 1
if success:
    print("Found solution in %d iterations with error: %s" % (it, error.T))
else:
    print("Could not find solution in %d iterations! Error: %s" % (it, error.T))
print(q.T)

In [ ]:
# Validation
fk_all(model, data, q)
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")

# target configuration!
fk_all(model, data, qd)
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")